<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_serperdev_demo_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook provides a detailed guide on leveraging Haystack 2.x and [serper.dev](https://serper.dev/) for implementing an OpenAPI service-based Retriever-Augmented Generation (RAG) workflow. It outlines an approach for processing user queries by integrating web search results into the context of Large Language Models (LLMs), enhancing their response generation.

The pipeline starts by fetching the OpenAPI specification for serper.dev and converting it into OpenAI function-calling definitions. serper.dev is then used as an OpenAPI service to retrieve relevant information from the web based on the user's input. The pipeline incorporates these search results, along with system and user prompts, into the LLM to enhance response generation.

The notebook will guide you through installing the necessary libraries, selecting an LLM provider, and constructing a Haystack 2.0 pipeline that orchestrates function calling, service requests, and LLM response generation.

Note: To run this pipeline, a [serper.dev](https://serper.dev/) account is required. Signing up is quick and provides you with 2,500 free queries—no credit card needed.

## 1. Setup

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [10]:
!pip install -q git+https://github.com/deepset-ai/haystack-experimental.git@openapi

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [11]:
import getpass
import os
import json
import requests

from typing import Dict, Any, List

from haystack import Pipeline
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.converters import OutputAdapter
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage, ByteStream
from haystack.utils import Secret

from haystack_experimental.components.tools.openapi import OpenAPITool, LLMProvider

## 2.  Enter the API keys for LLM provider and serper.dev

In [12]:
llm_api_key = getpass.getpass(f"Enter OpenAI api key for:")
serper_dev_key = getpass.getpass("Enter serperdev api key:")

Enter OpenAI api key for:··········
Enter serperdev api key:··········


## 3. Build our RAG serper.dev pipeline



In [13]:
pipe = Pipeline()
pipe.add_component("serperdev", OpenAPITool(generator_api=LLMProvider.OPENAI,
                                            generator_api_params={"model":"gpt-3.5-turbo",
                                                                  "api_key":Secret.from_token(llm_api_key)},
                                            tool_spec="https://bit.ly/serper_dev_spec_yaml",
                                            tool_credentials=serper_dev_key))

pipe.add_component("final_prompt_adapter", OutputAdapter("{{system_message + service_response}}", List[ChatMessage]))
pipe.add_component("llm", OpenAIChatGenerator(api_key=Secret.from_token(llm_api_key),
                                              generation_kwargs={"max_tokens": 1024},
                                              streaming_callback=print_streaming_chunk))


pipe.connect("serperdev", "final_prompt_adapter.service_response")
pipe.connect("final_prompt_adapter", "llm.messages")

🚅 Components
  - serperdev: OpenAPITool
  - final_prompt_adapter: OutputAdapter
  - llm: OpenAIChatGenerator
🛤️ Connections
  - serperdev.service_response -> final_prompt_adapter.service_response (List[ChatMessage])
  - final_prompt_adapter.output -> llm.messages (List[ChatMessage])

As you can see in the pipeline graph above, for a given serper.dev query, our pipeline follows these steps:

1. **Fetch OpenAPI Spec**: Retrieve the OpenAPI specification for serper.dev and convert it into OpenAI function-calling definitions.

2. **Determine Parameters**: Use a function-calling model to identify the necessary parameters for the serper.dev service.

3. **Query serper.dev**: Dispatch the request to serper.dev with the determined parameters and gather the responses.

4. **Compile Results**: Organize and format the search results from serper.dev.

5. **Generate Response**: Combine the system prompt, user query, and compiled search results, then pass them to LLM to generate the final response.

In [14]:
system_prompt = requests.get("https://bit.ly/serper_dev_system").text

In [20]:
query = "Who won mens Australian Open in 2024?"

result = pipe.run(data={"serperdev": {"messages": [ChatMessage.from_user(query)]},
                        "final_prompt_adapter": {"system_message": [ChatMessage.from_system(system_prompt)]}})


Jannik Sinner won the men's singles title at the 2024 Australian Open by defeating Daniil Medvedev in the final.